In [1]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
class Classifier(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.quant = torch.ao.quantization.QuantStub()
        self.model = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=2),
            nn.ReLU(),
            nn.Conv2d(8, 8*8, 3, stride=2),
            nn.ReLU(),
            nn.AvgPool2d(3, 2),
            nn.Flatten(),
            nn.Dropout(),
            nn.Linear(256, 128), 
            nn.Dropout(),
            nn.Linear(128, 10) 
            # nn.Softmax()
        )
        self.dequant = torch.ao.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        y = self.model(x)
        return self.dequant(y)
    

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to("cuda")
        y = y.to("cuda")
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [3]:
model = Classifier()

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
loss_fn = nn.CrossEntropyLoss()

In [4]:
model = model.to("cuda")
for i in range(100):
    train(train_dataloader, model, loss_fn, optimizer)
model = model.to("cpu")


loss: 2.312921  [   64/60000]
loss: 2.308734  [ 6464/60000]
loss: 2.291307  [12864/60000]
loss: 2.306224  [19264/60000]
loss: 2.318503  [25664/60000]
loss: 2.294582  [32064/60000]
loss: 2.308161  [38464/60000]
loss: 2.300468  [44864/60000]
loss: 2.304066  [51264/60000]
loss: 2.300486  [57664/60000]
loss: 2.308843  [   64/60000]
loss: 2.305158  [ 6464/60000]
loss: 2.296551  [12864/60000]
loss: 2.291258  [19264/60000]
loss: 2.317136  [25664/60000]
loss: 2.292890  [32064/60000]
loss: 2.309354  [38464/60000]
loss: 2.296918  [44864/60000]
loss: 2.306238  [51264/60000]
loss: 2.303803  [57664/60000]
loss: 2.301648  [   64/60000]
loss: 2.317602  [ 6464/60000]
loss: 2.292736  [12864/60000]
loss: 2.293813  [19264/60000]
loss: 2.318864  [25664/60000]
loss: 2.292829  [32064/60000]
loss: 2.306476  [38464/60000]
loss: 2.293043  [44864/60000]
loss: 2.302885  [51264/60000]
loss: 2.302706  [57664/60000]
loss: 2.292832  [   64/60000]
loss: 2.301071  [ 6464/60000]
loss: 2.268039  [12864/60000]
loss: 2.28

In [6]:
%%timeit
test(test_dataloader, model, loss_fn)

Test Error: 
 Accuracy: 74.6%, Avg loss: 0.711752 

Test Error: 
 Accuracy: 74.6%, Avg loss: 0.711752 

Test Error: 
 Accuracy: 74.6%, Avg loss: 0.711752 

Test Error: 
 Accuracy: 74.6%, Avg loss: 0.711752 

Test Error: 
 Accuracy: 74.6%, Avg loss: 0.711752 

Test Error: 
 Accuracy: 74.6%, Avg loss: 0.711752 

Test Error: 
 Accuracy: 74.6%, Avg loss: 0.711752 

Test Error: 
 Accuracy: 74.6%, Avg loss: 0.711752 

731 ms ± 256 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
model = model.to("cpu")

model.eval()
backend = "x86"
model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend

model_static_quantized = torch.quantization.prepare(model, inplace=False)
model_static_quantized = torch.quantization.convert(model_static_quantized, inplace=False)

/home/arthur/miniconda3/lib/python3.11/site-packages/torch/ao/quantization/observer.py:220: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/home/arthur/miniconda3/lib/python3.11/site-packages/torch/ao/quantization/observer.py:1272: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


In [9]:
%%timeit
test(test_dataloader, model_static_quantized, loss_fn)

Test Error: 
 Accuracy: 60.4%, Avg loss: 1.080245 

Test Error: 
 Accuracy: 60.4%, Avg loss: 1.080245 

Test Error: 
 Accuracy: 60.4%, Avg loss: 1.080245 

Test Error: 
 Accuracy: 60.4%, Avg loss: 1.080245 

Test Error: 
 Accuracy: 60.4%, Avg loss: 1.080245 

Test Error: 
 Accuracy: 60.4%, Avg loss: 1.080245 

Test Error: 
 Accuracy: 60.4%, Avg loss: 1.080245 

Test Error: 
 Accuracy: 60.4%, Avg loss: 1.080245 

933 ms ± 505 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
torch.save(model_static_quantized, "model_post_static_quantized.pt")